In [2]:
import twitter
from os.path import isfile
import pandas as pd
from time import sleep

In [3]:
api = twitter.Api(consumer_key='foYe2yl6efwHz9kJQiyI7vuSp',
                  consumer_secret="fITQUjsbsC938lb5VQpzFaCuW6d3LXhC5e34AkCtL5xkIvj1BD",
                  access_token_key='1110034001052471298-Bo7P9PV9bTlzIfxDT00DvW2TB4PsB8',
                  access_token_secret='JXh4LO37BLVqyiAhrXvwlh8riHe9EhTEnpqtAbXejz1WR')
api.InitializeRateLimit()

In [4]:
def twitter_search(title):    
    ratelim = twitter.ratelimit.RateLimit()
    remaining = ratelim.get_limit("https://api.twitter.com/1.1/application/rate_limit_status.json").remaining
    print(f'remaining = {remaining}')

    if remaining <= 0:
        print("failed")
        return False
    
    sub_dict = {'game_title': [], 'post_title': [], 'score': []}
    csv = f'game_titles_twitter.csv'

    # Set csv_loaded to True if csv exists since you can't evaluate the
    # truth value of a DataFrame.
    df, csv_loaded = (pd.read_csv(csv), 1) if isfile(csv) else ('', 0)

    print(f'csv = {csv}')
    print(f'csv_loaded = {csv_loaded}')
    print(f'Collecting information from Twitter for search term "{title}".')
    
    try:
        search = "q=" + title.replace(" ", "%20")
    except AttributeError:
        raise
    
    print(search)
    feed = api.GetSearch(raw_query=search)
    
    for post in feed:
        if post.retweeted_status is not None:
            notDuplicated = post.retweeted_status.text not in sub_dict['post_title']
            notCopied = post.retweeted_status.text not in tuple(df.post_title) if csv_loaded else True
            if notDuplicated and notCopied:
                print(post.retweeted_status.favorite_count)
                # Save any unique, non-stickied posts with descriptions to sub_dict.
                sub_dict['game_title'].append(title)
                sub_dict['post_title'].append(post.retweeted_status.text)
                sub_dict['score'].append(post.retweeted_status.favorite_count)
                sleep(0.1)

    # print(sub_dict)
    new_df = pd.DataFrame(sub_dict)

    # Add new_df to df if df exists then save it to a csv.
    if 'DataFrame' in str(type(df)):
        pd.concat([df, new_df], axis=0, sort=0).to_csv(csv, index=False)
        print(f'{len(new_df)} new posts collected and added to {csv}')
    else:
        new_df.to_csv(csv, index=False)
        print(f'{len(new_df)} posts collected and saved to {csv}')

In [5]:
twitter_search("Overwatch")

remaining = 15
csv = game_titles_twitter.csv
csv_loaded = 1
q=Overwatch
3
2
8
0
339
46
6 new posts collected and added to game_titles_twitter.csv


In [6]:
results = api.GetSearch(raw_query="q=twitter%20&result_type=recent&since=2014-07-19&count=100")

In [7]:
result = results[0]
print(result)

{"created_at": "Sun Mar 31 00:16:58 +0000 2019", "hashtags": [], "id": 1112146735130951680, "id_str": "1112146735130951680", "lang": "en", "media": [{"display_url": "pic.twitter.com/pRqwXrGxlI", "expanded_url": "https://twitter.com/SpursOfficial/status/1112049702206406661/video/1", "id": 1112049576624693250, "media_url": "http://pbs.twimg.com/media/D27MHrvXQAAEijx.jpg", "media_url_https": "https://pbs.twimg.com/media/D27MHrvXQAAEijx.jpg", "sizes": {"large": {"h": 1152, "resize": "fit", "w": 2048}, "medium": {"h": 675, "resize": "fit", "w": 1200}, "small": {"h": 383, "resize": "fit", "w": 680}, "thumb": {"h": 150, "resize": "crop", "w": 150}}, "type": "video", "url": "https://t.co/pRqwXrGxlI", "video_info": {"aspect_ratio": [16, 9], "duration_millis": 44840, "variants": [{"bitrate": 832000, "content_type": "video/mp4", "url": "https://video.twimg.com/amplify_video/1112049576624693250/vid/640x360/xqLCfXl0GJk2Z7nR.mp4?tag=11"}, {"bitrate": 2176000, "content_type": "video/mp4", "url": "htt

In [8]:
ratelim = twitter.ratelimit.RateLimit()
ratelim.get_limit("https://api.twitter.com/1.1/application/rate_limit_status.json")

EndpointRateLimit(limit=15, remaining=15, reset=0)

In [9]:
import unittest
import os

class TestTwitterMethods(unittest.TestCase):

    def test_bad_input(self):
        self.assertRaises(AttributeError, twitter_search, 123)
        
    def test_good_input(self):
        twitter_search("Overwatch")
        self.assertTrue(os.path.isfile('./game_titles_twitter.csv'))
 


In [10]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.

remaining = 15
csv = game_titles_twitter.csv
csv_loaded = 1
remaining = 15
csv = game_titles_twitter.csv
csv_loaded = 1
q=Overwatch
3
8
339


.

46
4 new posts collected and added to game_titles_twitter.csv



----------------------------------------------------------------------
Ran 2 tests in 0.577s

OK
